In [67]:
import openpyxl
import requests
import json

API_KEY = "API_KEY"
API_ENDPOINT = "https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze"

def evaluate_toxicity(comment):
    data = {
        'comment': {'text': comment},
        'languages': ['en'],
        'requestedAttributes': {'TOXICITY': {}},
    }
    params = {'key': API_KEY}

    response = requests.post(API_ENDPOINT, params=params, json=data)

    if response.ok:
        response_dict = json.loads(response.content)
        return response_dict['attributeScores']['TOXICITY']['summaryScore']['value']
    else:
        print(f"Error {response.status_code}: {response.reason}")
        return None

workbook = openpyxl.load_workbook("C:\\Users\\asiya\\Documents\\dataa.xlsx")
sheet = workbook.active

toxicity_scores = []
for row in sheet.iter_rows(min_row=2, values_only=True):
    comment_text = row[0]
    toxicity_score = evaluate_toxicity(comment_text)
    toxicity_scores.append(toxicity_score)

print(toxicity_scores)

[0.718943, 0.4014846, 0.85850734, 0.4574824, 0.89241093, 0.19314334, 0.03969839, 0.85333383, 0.47119883, 0.026970858, 0.0969861, 0.54823303, 0.9029226, 0.7105462, 0.37751234, 0.8988238, 0.033806015, 0.71760553, 0.9391453, 0.050573748, 0.7510937, 0.2854733, 0.64447093, 0.4269174, 0.2540519, 0.9391453, 0.6407703, 0.102192536, 0.5140397, 0.039462693, 0.36095104, 0.024849601, 0.8299589, 0.30527395, 0.5779532, 0.55739564, 0.45703048, 0.6744087, 0.5885171, 0.62136006, 0.6744087, 0.9061063, 0.47119883]


Hypothesis - Perspective API is more likely to make mistakes on toxic comments that use words that are less common in insults

I created separate dataframes out of the original dataset given. I used Perspective API to find the toxicity values for the values of each dataframes. After examining the scores of these dataframes of original data, I determined the threshold for toxicity to be 0.56; anything above this value is toxic.

I created a new dataset by taking a sample of comments from the original data. However, I changed singular words in some of the comments - I replaced words that commonly used for insults, with words that have the same meaning, but are less commonly used in negative ways. 

I found that my hypothesis was right. For example, while a comment including 'rape' was considered toxic, this same comment was not toxic when the word 'molest' was used in place of 'rape', despite both of these words referring to sexual abuse. Similarly, a comment with the word 'kill' was considered toxic, while this same comment was not found to be toxic when it used the word 'execute', despite both of these words refer to causing someone to die. Another instance: a comment had a toxicity above the threshold when 'sex' was used but not when 'intercourse' was put in place of 'sex'.

I think this bias exists because of the connotations behind certain words. Certain words may have a more proper or formal connotation - they may be used in more professional settings, making them seem less insulting or rude. Other words may have the same exact meaning but are used much more casually and crudely, due to the connotations that society has given them. People may use these crude words to have the intended effect of harassing or insulting with their toxic comment. So, a machine may be trained to more easily catch these words that have more negative connotations, as these may seem more harmful, even if they have the exact same meaning as another word.

This is a relevant and important problem, as it could give people a way to insult or harass others without getting caught. For example, take one of my findings, where Perspective API only scored a comment as having a toxicity above the threshold when it involved the word 'kill', but not when 'kill' was replaced with 'execute'. Someone could make a death threat using the word 'execute' instead of 'kill', as a way to avoid getting caught by a machine. It is important to address this to ensure that all toxic comments are eliminated, despite their wording.